In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classifying-seti-data-from-radio-signals/Project/spec_augment.py
/kaggle/input/classifying-seti-data-from-radio-signals/Project/Guided Project - Classify Radio Signals using PyTorch.ipynb
/kaggle/input/classifying-seti-data-from-radio-signals/Project/valid.csv
/kaggle/input/classifying-seti-data-from-radio-signals/Project/image6.png
/kaggle/input/classifying-seti-data-from-radio-signals/Project/utils.py
/kaggle/input/classifying-seti-data-from-radio-signals/Project/Untitled-design.png
/kaggle/input/classifying-seti-data-from-radio-signals/Project/train.csv
/kaggle/input/classifying-seti-data-from-radio-signals/Project/sparse_image_warp.py


# **Introdution**

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import PyTorch and related modules
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# Import torchvision transformations
from torchvision import transforms as T

# Import a popular image model library
import timm

# **First Image**